In [9]:
sandbox = False
get_access_only = False
access_code = "8914"

if sandbox==False:
    userid = "NT1945"
    password='BT121299@'
    access_token = "3bf7af61-77c2-3f2e-b1c1-f4f5cb909db3"
    consumer_key = "ay4bObamN3KHOPzZjmpA7g9YDJwa"
    host = 'https://tradeapi.kotaksecurities.com/apim'
else:
    userid = "1T068"
    password='login@1'
    access_token = "ebe9c833-26c1-3402-bf80-c457e76a4da3"
    consumer_key = "a0s3dcOM4JgVNn3pB7fdLBFkJ4Ea"
    host = 'https://sbx.kotaksecurities.com/apim'

In [10]:
#GENERATING OTP
from ks_api_client.ks_api import KSTradeApi
client = KSTradeApi(access_token = access_token, userid = userid, \
                consumer_key = consumer_key, ip = "127.0.0.1", app_id = "DefaultApplication", host = host)
client.login(password=password)

{'Success': {'Status': 'Temporary',
  'accessCodeTime': '12/15/2021 12:18:41 PM',
  'authLevel': 2,
  'biometric': 'N',
  'clientType': 'ONLINE',
  'emailId': 'maXXXXXXXXXXXXXXXse@gmail.com',
  'message': 'Authentication Successful.',
  'mpin': 'N',
  'nomineeFlag': 0,
  'oneTimeToken': '1B278DB9C1454D70A4C2DB2EA9FE43F7',
  'phoneNo': 'XXXXXXX123',
  'redirect': {'host': 'https://tradeapi.kotaksecurities.com/apim',
   'port': 443},
  'userid': 'NT1945'}}

In [11]:
#OTP verification

if get_access_only==True:
    exit()

if sandbox==True:
    client.session_2fa(access_code = '1111')
else:
    client.session_2fa(access_code = access_code)

In [14]:
#Get token list

from datetime import datetime
import requests
fno_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_'+datetime.now().strftime("%d_%m_%Y")+'.txt'
cash_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_CASH_'+datetime.now().strftime("%d_%m_%Y")+'.txt'

#fno_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_10_12_2021.txt'
#cash_url = 'https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_CASH_10_12_2021.txt'

fno_tokens = requests.get(fno_url).text
cash_tokens = requests.get(cash_url).text

fno_tokens = fno_tokens.split('\n')
fno_tokens = [i.strip().split('|') for i in fno_tokens]

cash_tokens = cash_tokens.split('\n')
cash_tokens = [i.strip().split('|') for i in cash_tokens]

print(len(fno_tokens))
print(len(cash_tokens))

103347
7232


In [15]:
if len(fno_tokens) <= 1 or len(cash_tokens) <= 1:
    assert True==False, "Token list not valid"

In [16]:
import pandas as pd

fno_df = pd.DataFrame( fno_tokens[1:], columns=fno_tokens[0])
cash_df = pd.DataFrame( cash_tokens[1:], columns=cash_tokens[0])

In [17]:
fno_df.head()

,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,optionType
0,31325,CRDDECFE22,,0,17DEC21,0,1.000000,100,CF,COMMODITY,MCX-CM,,1,234512,XX
1,33213,CRDDECJA22,,0,17DEC21,0,1.000000,100,CF,COMMODITY,MCX-CM,,1,234199,XX
2,20323,CRUDEOIL,,5438,17DEC21,0,1.000000,100,CF,COMMODITY,MCX-CM,,1,230697,XX
3,18829,NATURALGAS,,297.4,28DEC21,0,0.100000,1250,CF,COMMODITY,MCX-CM,,1,233491,XX
4,18347,ALUDECFE22,,0,31DEC21,0,0.050000,5,CF,COMMODITY,MCX-CM,,1000,234638,XX


In [18]:
cash_df.head()

,instrumentToken,instrumentName,name,lastPrice,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,isin,multiplier,exchangeToken,OptionType
0,9516,20MICRONS,20 Microns Ltd,62.3,0,0,0.050000,1,EQ,CASH,NSE,INE144J01027,1,16921,-
1,43865,3IINFOLTD,3i Infotech Ltd,111.1,0,0,0.050000,1,EQ,CASH,NSE,INE748C01038,1,6232,-
2,891,3MINDIA,3M India Ltd,24777.5,0,0,0.050000,1,EQ,CASH,NSE,INE470A01017,1,474,-
3,607,3PLAND,3P Land Holdings Ltd,14.4,0,0,0.050000,1,EQ,CASH,NSE,INE105C01023,1,2595,-
4,13140,5PAISA,5Paisa Capital Ltd,429.25,0,0,0.050000,1,EQ,CASH,NSE,INE618L01018,1,445,-


In [19]:
#Get cash tokens

banknifty_token = int(cash_df[cash_df.instrumentName=='NIFTY BANK'].instrumentToken.values[0])
print("BANK NIFTY TOKEN:",banknifty_token)

BANK NIFTY TOKEN: 11717


In [30]:
# get ltp and atm

def get_atm():
    banknity_ltp = client.quote(instrument_token = banknifty_token, quote_type = "LTP")
    if 'success' in banknity_ltp:
        banknity_ltp = banknity_ltp['success'][0]['lastPrice']
    else:
        assert True==False

    banknity_ltp = float(banknity_ltp)
    banknity_atm = round(banknity_ltp, -2)

    print('BANK NIFTY ATM:', banknity_atm)
    return int(banknity_atm)

banknity_atm = get_atm()

BANK NIFTY ATM: 36800.0


In [32]:
# get pe ce tokens for a strike
def get_pe_ce_token(banknity_atm, fno_df):
    fno_df = fno_df[ (fno_df.instrumentName == 'BANKNIFTY') & (fno_df.strike==str(int(banknity_atm)))][0:2]
    banknity_token_pe = int(fno_df[fno_df.optionType=='PE']['instrumentToken'].values[0])
    banknity_token_ce = int(fno_df[fno_df.optionType=='CE']['instrumentToken'].values[0])

    print(f"BANK NIFTY {banknity_atm} PE TOKEN:",banknity_token_pe)
    print(f"BANK NIFTY {banknity_atm} CE TOKEN:",banknity_token_ce)
    return banknity_token_pe,banknity_token_ce

banknity_token_pe, banknity_token_ce = get_pe_ce_token(banknity_atm, fno_df)
banknity_token_pe, banknity_token_ce

BANK NIFTY 36800 PE TOKEN: 39120
BANK NIFTY 36800 CE TOKEN: 39119


(39120, 39119)

In [34]:
def get_quote(instrument_token):
    ltp = client.quote(instrument_token = instrument_token, quote_type = "LTP")
    if 'success' in ltp:
        ltp = ltp['success'][0]['lastPrice']
    else:
        assert True==False

    return ltp

print(f'banknity_ce {banknity_atm} LTP', get_quote(banknity_token_ce))
print(f'banknity_pe {banknity_atm} LTP', get_quote(banknity_token_pe))

banknity_ce 36800 LTP 244.3500
banknity_pe 36800 LTP 208.4500


In [24]:
from datetime import datetime

old_second = 70

while True:
    
    hour, minute, second = [int(i) for i in datetime.now().strftime("%H.%M.%S").split('.')]
    
    if(old_second!=second):
        print(hour,minute,second)

        #update time
        old_second=second
        
        #Before 9:30am
        if(hour==9 and minute==29 and int(second)==58):
            #get banknifty atm
            banknity_atm = get_atm()
            
            #get pe and ce token at that atm
            banknity_token_pe, banknity_token_ce = get_pe_ce_token(banknity_atm, fno_df)

            #initialize hit flags
            banknifty_ce_hit = False
            banknifty_pe_hit = False
            banknifty_ce_wrong = False
            banknifty_pe_wrong = False

        #At 9:30am
        if(hour==9 and minute==30 and int(second)==0):
            #get ce and pe quote
            banknifty_ce_ltp_930 = get_quote(banknity_token_ce)
            banknifty_pe_ltp_930 = get_quote(banknity_token_pe)

            banknifty_ce_ex = 0.9*banknifty_ce_ltp_930
            banknifty_pe_ex = 0.9*banknifty_pe_ltp_930
        
        #After 9:30am
        if(hour>=9 and minute>=30):
            banknifty_ce_ltp = get_quote(banknity_token_ce)
            banknifty_pe_ltp = get_quote(banknity_token_pe)

            if (banknifty_ce_ltp<=banknifty_ce_ex and banknifty_ce_hit==False):
                #sell ce
                banknifty_ce_hit = True
                pass

            if (banknifty_ce_hit==True and banknifty_ce_ltp>=1.25*banknifty_ce_ex):
                #buy ce
                banknifty_ce_wrong = True
                pass

            if (banknifty_pe_ltp<=banknifty_pe_ex and banknifty_pe_hit==False):
                #sell pe
                banknifty_pe_hit = True
                pass

            if (banknifty_pe_hit==True and banknifty_pe_ltp>=1.25*banknifty_pe_ex):
                #buy pe
                banknifty_pe_wrong = True
                pass

            #At 3pm
            if(hour==15 and minute==0 and int(second)==0):
                if( banknifty_ce_hit==True and banknifty_ce_wrong==False ):
                    #buy ce
                    pass

                if( banknifty_pe_hit==True and banknifty_pe_wrong==False ):
                    #buy pe
                    pass            

        

        

2 24 40
2 24 41
2 24 42
2 24 43
2 24 44
2 24 45
2 24 46
2 24 47
2 24 48
2 24 49
2 24 50
2 24 51
2 24 52
2 24 53
2 24 54
2 24 55
2 24 56
2 24 57
2 24 58
2 24 59
2 25 0
BANK NIFTY ATM: 37100
BANK NIFTY 37100 PE TOKEN: 39126
BANK NIFTY 37100 CE TOKEN: 39125
2 25 1
2 25 2
2 25 3


KeyboardInterrupt: 